In [90]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import dateutil.parser
from pprint import pprint
%matplotlib inline

In [5]:
divvy_df = pd.read_csv('Divvy_Trips_2014-Q3-07.csv', sep = ',')

In [19]:
divvy_df.head()

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,2886259,7/31/2014 23:56,8/1/2014 0:03,2602,386,291,Wells St & Evergreen Ave,53,Wells St & Erie St,Subscriber,Female,1979
1,2886258,7/31/2014 23:58,8/1/2014 0:07,2403,495,98,LaSalle St & Washington St,106,State St & Pearson St,Subscriber,Male,1974
2,2886257,7/31/2014 23:58,8/1/2014 2:10,669,7947,240,Sheridan Rd & Irving Park Rd,240,Sheridan Rd & Irving Park Rd,Customer,NaN,NaN
3,2886256,7/31/2014 23:58,8/1/2014 0:19,2431,1282,47,State St & Kinzie St,14,Morgan St & 18th St,Customer,NaN,NaN
4,2886255,7/31/2014 23:57,8/1/2014 2:10,2885,7972,240,Sheridan Rd & Irving Park Rd,240,Sheridan Rd & Irving Park Rd,Customer,NaN,NaN


In [21]:
divvy_hr_df = divvy_df.copy(deep = True)

In [28]:
def hourbinner(fulltime):
    return dateutil.parser.parse(fulltime).replace(minute=0, second=0)
    
# print dateutil.parser.parse('7/31/2014 23:58').replace(minute=0, second=0)

In [29]:
divvy_hr_df.starttime = divvy_hr_df.starttime.apply(lambda x: hourbinner(x))

In [32]:
divvy_hr_df.stoptime = divvy_hr_df.stoptime.apply(lambda x: hourbinner(x))

In [49]:
print divvy_hr_df.groupby(['from_station_name'])['starttime'].size().order(ascending = False)

from_station_name
Streeter Dr & Illinois St       11149
Lake Shore Dr & Monroe St        8533
Theater on the Lake              8358
Michigan Ave & Oak St            7338
Lake Shore Dr & North Blvd       6872
Millennium Park                  5560
Clinton St & Washington Blvd     5237
Canal St & Adams St              4840
Columbus Dr & Randolph St        4672
Museum Campus                    4670
Canal St & Madison St            4336
McClurg Ct & Illinois St         3889
Michigan Ave & Washington St     3794
Clark St & Elm St                3760
Indiana Ave & Roosevelt Rd       3545
Wells St & Concord Ln            3438
Michigan Ave & Jackson Blvd      3339
Daley Center Plaza               3228
Franklin St & Arcade Pl          3211
State St & Erie St               3208
Lake Shore Dr & Ohio St          3179
Dearborn St & Adams St           3177
Michigan Ave & Lake St           3113
Clark St & Lincoln Ave           3090
Michigan Ave & Pearson St        3066
State St & Kinzie St            

In [146]:
fromstation_starttime = divvy_hr_df.groupby(['from_station_name', 'starttime']).size()

In [147]:
ts = fromstation_starttime.unstack(level=0)

In [149]:
index = pd.date_range('2014-07-01', '2014-07-31', freq='H')

In [150]:
ts = ts.reindex(index, fill_value = 0)

In [154]:
ts = ts.fillna(0)

In [156]:
ts = ts.stack()

In [157]:
ts = ts.swaplevel(0,1)

In [158]:
ts = ts.sortlevel()

In [165]:
tostation_stoptime = divvy_hr_df.groupby(['to_station_name', 'stoptime']).size()

In [166]:
vs = tostation_stoptime.unstack(level=0)

In [167]:
vs = vs.reindex(index, fill_value = 0)

In [168]:
vs = vs.fillna(0)

In [169]:
vs = vs.stack()

In [170]:
vs = vs.swaplevel(0,1)

In [171]:
vs = vs.sortlevel()

In [200]:
combined = pd.concat([ts, vs], axis=1)

In [201]:
combined.columns = ['outflow', 'inflow']

In [202]:
combined.head()

outflow  inflow
from_station_name                                     
900 W Harrison    2014-07-01 00:00:00        0       0
                  2014-07-01 01:00:00        0       0
                  2014-07-01 02:00:00        0       0
                  2014-07-01 03:00:00        0       0
                  2014-07-01 04:00:00        0       0

In [205]:
combined['flowdiff'] = combined['inflow'] - combined['outflow']

In [207]:
combined

outflow  inflow  flowdiff
from_station_name                                                    
900 W Harrison         2014-07-01 00:00:00        0       0         0
                       2014-07-01 01:00:00        0       0         0
                       2014-07-01 02:00:00        0       0         0
                       2014-07-01 03:00:00        0       0         0
                       2014-07-01 04:00:00        0       0         0
                       2014-07-01 05:00:00        0       0         0
                       2014-07-01 06:00:00        0       0         0
                       2014-07-01 07:00:00        0       0         0
                       2014-07-01 08:00:00        1       3         2
                       2014-07-01 09:00:00        0       2         2
                       2014-07-01 10:00:00        1       2         1
                       2014-07-01 11:00:00        1       0        -1
                       2014-07-01 12:00:00        1       0        -1
                       2014-07-01 13:00:00        1       0        -1
                       2014-07-01 14:00:00        0       1         1
                       2014-07-01 15:00:00        0       0         0
                       2014-07-01 16:00:00        1       0        -1
                       2014-07-01 17:00:00        3       0        -3
                       2014-07-01 18:00:00        0       1         1
                       2014-07-01 19:00:00        0       1         1
                       2014-07-01 20:00:00        0       0         0
                       2014-07-01 21:00:00        0       0         0
                       2014-07-01 22:00:00        1       1         0
                       2014-07-01 23:00:00        0       0         0
                       2014-07-02 00:00:00        0       0         0
                       2014-07-02 01:00:00        1       0        -1
                       2014-07-02 02:00:00        0       0         0
                       2014-07-02 03:00:00        0       0         0
                       2014-07-02 04:00:00        0       0         0
                       2014-07-02 05:00:00        0       0         0
...                                             ...     ...       ...
Woodlawn Ave & 55th St 2014-07-29 19:00:00        0       0         0
                       2014-07-29 20:00:00        0       0         0
                       2014-07-29 21:00:00        0       0         0
                       2014-07-29 22:00:00        3       0        -3
                       2014-07-29 23:00:00        0       0         0
                       2014-07-30 00:00:00        0       0         0
                       2014-07-30 01:00:00        0       0         0
                       2014-07-30 02:00:00        0       0         0
                       2014-07-30 03:00:00        0       0         0
                       2014-07-30 04:00:00        0       0         0
                       2014-07-30 05:00:00        0       0         0
                       2014-07-30 06:00:00        0       0         0
                       2014-07-30 07:00:00        0       0         0
                       2014-07-30 08:00:00        2       1        -1
                       2014-07-30 09:00:00        1       0        -1
                       2014-07-30 10:00:00        0       4         4
                       2014-07-30 11:00:00        0       2         2
                       2014-07-30 12:00:00        0       0         0
                       2014-07-30 13:00:00        1       0        -1
                       2014-07-30 14:00:00        0       1         1
                       2014-07-30 15:00:00        1       0        -1
                       2014-07-30 16:00:00        2       1        -1
                       2014-07-30 17:00:00        0       1         1
                       2014-07-30 18:00:00        1       1         0
                       2014-07-30 19:00:00        3   